In [1]:
import sparknlp

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as fun
from pyspark.sql.types import *

import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

from sparknlp.pretrained import PretrainedPipeline
from sparknlp import Finisher
#Spark version and Spark NLP version should be align

In [2]:
spark = SparkSession.builder \
    .appName("nlp")\
    .master("local[*]")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.4")\
    .getOrCreate()

22/06/02 11:43:42 WARN Utils: Your hostname, winware resolves to a loopback address: 127.0.1.1; using 172.25.24.223 instead (on interface eth0)
22/06/02 11:43:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/rjac/anaconda3/envs/py-sparknlp/lib/python3.10/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/rjac/.ivy2/cache
The jars for the packages stored in: /home/rjac/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4fcb98c5-52dc-4e10-8604-b0c0a2c38a1b;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.4 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1

In [3]:
path = "/home/rjac/workspace/wikipedia-biography-dataset/wikipedia-biography-dataset/train/train.sent"

In [4]:
df = spark.read.text(path)

In [5]:
df.show(5,vertical=True,truncate=350)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 value | walter extra is a german award-winning aerobatic pilot , chief aircraft designer and founder of extra flugzeugbau -lrb- extra aircraft construction -rrb- , a manufacturer of aerobatic aircraft . 
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 value | extra was trained as a mechanical engineer .                                                                                                                                                       
-RECORD 2---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
pipeline = PretrainedPipeline('explain_document_ml', lang='en')

explain_document_ml download started this may take some time.
Approx size to download 9.1 MB
[ | ]explain_document_ml download started this may take some time.
Approximate size to download 9.1 MB
Download done! Loading the resource.
[ / ]

[ — ]

[ \ ]

[ / ]

[OK!]


In [7]:
docs = pipeline.annotate(df,"value")

In [8]:
docs.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentence: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true

In [9]:
extractor = Finisher()

In [10]:
extractor = extractor.setInputCols(["spell","pos"])
extractor = extractor.setOutputAsArray(True)

In [11]:
preproceed_df = extractor.transform(docs)

In [12]:
preproceed_df.show(5,vertical=True,truncate=150)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------
 text           | walter extra is a german award-winning aerobatic pilot , chief aircraft designer and founder of extra flugzeugbau -lrb- extra aircraft construction... 
 finished_spell | [waiter, extra, is, a, germane, awardwinning, aerobatic, pilot, ,, chief, aircraft, designer, and, founder, of, extra, flugzeugbau, -, lrb, -, extr... 
 finished_pos   | [NN, NN, VBZ, DT, NN, VBG, JJ, NN, ,, JJ, NN, NN, CC, NN, IN, JJ, NN, -, NN, -, JJ, NN, NN, -, NN, -, ,, DT, NN, IN, JJ, NN, .]                        
-RECORD 1----------------------------------------------------------------------------------------------------------------------------------------------------------------
 text           | extra was trained as a mechanical engineer .                                                                                        

In [13]:
preproceed_df.write.json("data")